# Student Performance Analysis & Prediction

In this notebook, we will explore a dataset containing various factors influencing student performance and build a machine learning model using PyTorch to predict student grades based on these factors.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, accuracy_score


ModuleNotFoundError: No module named 'torch'

## 1. Load the Dataset

Let's load the student performance dataset from Kaggle.

In [ ]:
# Load the dataset
df = pd.read_csv('path_to_your_dataset.csv')  # Provide the correct path to your dataset
df.head()

## 2. Exploratory Data Analysis (EDA)

We will now explore the data, check for missing values, and visualize the relationships between different variables.

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Describe the data
df.describe()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Visualizing relationships between key variables (e.g., study time and grades)
sns.pairplot(df[['studytime', 'absences', 'G1', 'G2', 'G3']])
plt.show()

## 3. Data Preprocessing

We will now preprocess the data, including handling categorical variables and scaling numeric features.

In [ ]:
# Convert categorical variables into dummy/one-hot encoded variables
df = pd.get_dummies(df, drop_first=True)

# Select features and target variable
X = df.drop(['G3'], axis=1)  # G3 is the final grade we want to predict
y = df['G3']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## 4. Building a Neural Network with PyTorch

We will now create a simple feedforward neural network using PyTorch to predict student performance.

In [ ]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define the neural network model
class StudentPerformanceNN(nn.Module):
    def __init__(self, input_size):
        super(StudentPerformanceNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function and optimizer
input_size = X_train.shape[1]
model = StudentPerformanceNN(input_size)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training the model
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

## 5. Model Evaluation

Now, let's evaluate our model on the test data.

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = loss_fn(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Calculate mean squared error
mse = mean_squared_error(y_test_tensor, predictions)
print(f'Mean Squared Error: {mse:.4f}')

## 6. Conclusion

We successfully explored the student performance dataset, built a neural network model using PyTorch, and evaluated its performance. Further improvements could involve hyperparameter tuning and exploring more advanced models.